In [1]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [2]:
servername = 'LAPTOP-RPEMJO16\\SQLEXPRESS'

# Northwind connectie

In [3]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [4]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [5]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [6]:
aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
nwtest = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\2905875312.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\2905875312.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\2905875312.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nwtest = pd.read_sql_query('SELECT * FROM cu

In [36]:
# Klopt echt helemaal niets van de aenc - > nw orders draw io

# Select NorthWind
nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
nw_OrderDetails = pd.read_sql_query('SELECT * FROM [Order Details]', northwind_conn)
# Select Adventureworks
Prod_Prod = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
Prod_Inv = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)

Sales_Sales = pd.read_sql_query('SELECT * FROM Sales.SalesOrderDetail', adventureworks_conn)
Sales_OrderHeader = pd.read_sql_query('SELECT * FROM Sales.SalesOrderHeader', adventureworks_conn)


# Select aenc
AE_Customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
AE_SalesOrderItem = pd.read_sql_query('SELECT * FROM sales_order_item', aenc_conn)
AE_SalesOrder = pd.read_sql_query('SELECT * FROM sales_order', aenc_conn)
AE_SalesOrderMerged = AE_SalesOrder.merge(AE_SalesOrderItem, on='id', how='right')
# aenc customer + nw employee ->    nw orders
MergeNwOrders1 = AE_Customer.merge(nw_Ord, left_on='id', right_on='CustomerID', how='right')
MergeNwOrders2 = nw_Emp.merge(MergeNwOrders1, on='EmployeeID', how='right')


MergedProdProd = Prod_Inv.merge(Prod_Prod, how='outer')

NWorderDetails1 = MergeNwOrders2.merge(nw_OrderDetails, on='OrderID', how='right')
NWorderDetailsFinal = MergedProdProd.merge(NWorderDetails1, how='outer')

OrderDetails = NWorderDetailsFinal[['OrderID', 'ProductID', 'Quantity']]
OrderDetails.dropna()

C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\3311397764.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Emp = pd.read_sql_query('SELECT * FROM Employees', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\3311397764.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_Ord = pd.read_sql_query('SELECT * FROM Orders', northwind_conn)
C:\Users\kwand\AppData\Local\Temp\ipykernel_15844\3311397764.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_OrderDetails = pd.read_sql_query('SELECT * FROM [O

,OrderID,ProductID,Quantity
0,11005.0,1,2.0
1,10609.0,1,3.0
2,10838.0,1,4.0
3,11003.0,1,4.0
4,10700.0,1,5.0
...,...,...,...
2166,10886.0,77,40.0
2167,10946.0,77,40.0
2168,10451.0,77,55.0
2169,10590.0,77,60.0
